In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import json
from nltk.tokenize import  word_tokenize
import tensorflow as tf
import ast

/home/pratik/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Import word embeddings and reformat
import gensim
from gensim.models.keyedvectors import KeyedVectors
word_vectors = gensim.models.Word2Vec.load('nearby_embeddings2')
wordVectors = []
for word in word_vectors.wv.index2word:
    wordVectors.append(word_vectors.wv[word])
wordVectors = np.array(wordVectors)
wordsList = word_vectors.wv.index2word

max_sequence_length = 20

In [3]:
raw_data = pd.read_csv('/home/pratik/NER/data/large_positive.csv')
len(raw_data)

21682

In [4]:
raw_data.drop_duplicates(subset=['body'], inplace=True)
len(raw_data)

16623

In [5]:
def convert_to_list(original_text_list):
    l = ast.literal_eval(original_text_list)
    l = [i.strip() for i in l]
    return l
raw_data['original_text'] = raw_data['original_text'].apply(lambda x: convert_to_list(x))

In [7]:
def generate_y(text, original_text_list, k):
    y = np.zeros([max_sequence_length, 2])
    try:
        for i in range(len(y)):
            y[i][0] = 1
        for original_text_words in original_text_list:
            for original_text in word_tokenize(original_text_words):
                words = word_tokenize(text)
                for i in range(len(words)):
                    if words[i].lower() == original_text.lower():
                      #  print(words[i], original_text)
                        y[i][1] = 1
                        y[i][0] = 0
    except:
        pass
    return y


In [8]:
y_list = np.zeros([len(raw_data), max_sequence_length, 2])
for body, original_text_list, i in zip(raw_data['body'], raw_data['original_text'], range(len(raw_data))):
    
    y_list[i] = generate_y(body, original_text_list, i)

In [9]:
word_tokenize(raw_data['body'].iloc[11]), raw_data['original_text'].iloc[11], y_list[11]

(['No', 'I', 'ask', 'Paytm', 'debit', 'card', 'queries'],
 ['paytm debit'],
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]]))

In [10]:
ids = np.load('nearby_ids2.npy')

In [11]:
char_vocab = []
def get_character_set(text):
    for char in text:
        if not char in char_vocab:
            char_vocab.append(char)

raw_data['body'].apply(lambda x: get_character_set(x))

0        None
1        None
2        None
3        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
30       None
         ... 
21287    None
21288    None
21289    None
21290    None
21291    None
21292    None
21293    None
21294    None
21295    None
21296    None
21297    None
21298    None
21299    None
21301    None
21302    None
21304    None
21305    None
21306    None
21307    None
21309    None
21310    None
21311    None
21312    None
21313    None
21316    None
21318    None
21319    None
21320    None
21321    None
21322    None
Name: body, Length: 16623, dtype: object

In [12]:
char_vocab_len = len(char_vocab)

In [13]:
char_one_hot = np.zeros([char_vocab_len, char_vocab_len])
for i in range(len(char_vocab)):
    char_one_hot[i][i] = 1
    

In [14]:
max_char_length = 16
max_sequence_length = 20
char_dimensions = char_vocab_len
batch_size = 100
raw_data_len = len(raw_data)
ids_char = np.zeros([(raw_data_len),max_sequence_length,max_char_length ])
ids_char = ids_char + len(char_vocab) -1

In [15]:
g = 0
for i in (range((raw_data_len))):
    body = raw_data.iloc[i]['body']
    words = word_tokenize(body)
    for j in (range(len(words))):
        word = words[j]
        for k in range(len(word)):
            char = word[k]
            try:
                ids_char[i][j][k] = char_vocab.index(char)
            except:
                pass

In [16]:
ids_char.shape

(16623, 20, 16)

In [17]:
import tensorflow as tf
tf.reset_default_graph()


In [18]:
#weights = {'W_conv1': tf.get_variable("W_conv1", shape=([5,5,1,32]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'W_conv2':                tf.get_variable("W_conv2", shape=([5,5,32,64]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'W_fc':tf.get_variable("W_fc", shape=([(int(max_char_length//4))*(int(char_dimensions//4))*64,1024]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer())
#           }



#biases = {'b_conv1':tf.get_variable("b_conv1", shape=([32]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'b_conv2':tf.get_variable("b_conv2", shape=([64]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'b_fc':tf.get_variable("b_fc", shape=([1024]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer())
#           }
weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),
           'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
           'W_fc':tf.Variable(tf.random_normal([(int(max_char_length//2))*(int(char_dimensions//2))*32,100])),
           'out':tf.Variable(tf.random_normal([100, 10]))}

biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
           'b_conv2':tf.Variable(tf.random_normal([64])),
           'b_fc':tf.Variable(tf.random_normal([100])),
           'out':tf.Variable(tf.random_normal([10]))}

def character_level_embedding(char_input_data):
    print('sdiugfh', char_input_data)
    char_input_data = tf.cast(char_input_data, dtype=tf.int32)
    char_one_hot_tensor = tf.convert_to_tensor(char_one_hot, dtype=tf.int32)
    char_data = tf.placeholder(shape = (max_sequence_length,max_char_length), dtype=tf.int32)
    char_data = tf.nn.embedding_lookup(char_one_hot_tensor,char_input_data)
    print('char_data', char_data)
    keep_rate = 1
    print('aya')
    n_nodes_hl1 = 100
    #print(char_data.shape)
#The place holder to hold the values of new x and y at different times.
    x = tf.placeholder(dtype=tf.float32, shape=[max_sequence_length, max_char_length * char_dimensions])
    x = tf.cast(tf.reshape(char_data, shape=[max_sequence_length, max_char_length * char_dimensions]), dtype=tf.float32)
    #print(x)
#y = tf.placeholder('float', [batch_size, n_classes])

    def embedding_layer(data):
        print('aya')
        hidden_1_layer = {'weights':tf.Variable(tf.random_normal([max_char_length * char_dimensions, n_nodes_hl1])),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
        l1 = tf.add(tf.tensordot(data,hidden_1_layer['weights'], axes=1), hidden_1_layer['biases'])
        l1 = tf.nn.relu(l1)
        print(l1.shape)
        return l1

    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

    def maxpool2d(x):
        #                        size of window         movement of window
        return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


    def convolutional_neural_network(x):
        
#        tf.get_variable("W_conv1", shape=([5,5,1,32]), dtype=tf.int32,
#                      initializer=tf.ones_initializer())
#                tf.get_variable("W_conv2", shape=([5,5,32,64]), dtype=tf.int32,
#                      initializer=tf.ones_initializer())
#        
#                        tf.get_variable("W_fc", shape=([(int(max_char_length//4))*(int(char_dimensions//4))*64,1024]), dtype=tf.int32,
#                      initializer=tf.ones_initializer())

        
        

        print('og', x.shape)
        x = tf.reshape(x, shape=[max_sequence_length,max_char_length,char_dimensions, 1])

        conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
        conv2 = maxpool2d(conv1)

        #conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
        #conv2 = maxpool2d(conv2)
        
        print('con2',conv2.shape)
        fc = tf.reshape(conv2,[max_sequence_length, (max_char_length//2) * (char_dimensions//2) * 32])
        fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
        fc = tf.nn.dropout(fc, keep_rate)
        #print(fc.shape)
        return fc

    def train_neural_network(x):    
        #embedded_x = embedding_layer(x)
        #embedded_x = tf.reshape(x, [max_sequence_length, char_dimensions])
        convoluted_x = convolutional_neural_network(x)
        
        return convoluted_x
    
    char_data = train_neural_network(x)
    return char_data


char_input_data = tf.placeholder(dtype=tf.float32,shape= [batch_size, max_sequence_length, max_char_length])
char_embeddings = tf.map_fn(lambda x: character_level_embedding(x), char_input_data, dtype=tf.float32)



sdiugfh Tensor("map/while/TensorArrayReadV3:0", shape=(20, 16), dtype=float32)
char_data Tensor("map/while/embedding_lookup:0", shape=(20, 16, 158), dtype=int32)
aya
og (20, 2528)
con2 (20, 8, 79, 32)


In [19]:
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
with tf.Session() as sess:
    sess.run(init_g)
    sess.run(init_l)
    p = ids_char.astype(int)[100:200]
    print(type(p[0][0][0]))
    b = sess.run([char_embeddings], feed_dict={char_input_data: p})
    print(b[0].shape)
    

<class 'numpy.int64'>
(100, 20, 100)


In [20]:
lstmUnits = 64
numClasses = 2
epochs = 10
numDimensions = 100
batchSize = 100
train_len = 14000

X_train = ids[:train_len]
y_train = y_list[:train_len]
X_test = ids[train_len:]
y_test = y_list[train_len:]
ids_char_train = ids_char[:train_len]
ids_char_test = ids_char[train_len:]




#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test, ids_char_train, ids_char_test = train_test_split(ids,y_list,ids_char, test_size=.2)

len(X_train), len(y_test), len(ids_char_train)

(14000, 2623, 14000)

In [21]:
def getTrainBatch(j):
    i = j*batchSize
    arr = X_train[i:i+batchSize]
    labels = y_train[i:i+batchSize]
    arr_char = ids_char_train[i:i+batchSize]
    #print(labels[0])
    return arr, labels, arr_char

In [22]:
def getTestBatch(j):
    i = j*batchSize
    arr = X_test[i:i+batchSize]
    labels =   y_test[i:i+batchSize]
    arr_char = ids_char_test[i:i+batchSize]
    return arr, labels, arr_char

In [23]:
labels = tf.placeholder(tf.float32, [batchSize, max_sequence_length, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, max_sequence_length])

In [24]:
wordVectors = tf.convert_to_tensor(wordVectors, dtype=tf.float32)

In [25]:
data = tf.placeholder(shape = (batchSize,max_sequence_length,numDimensions), dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)
print(data.shape)
print(char_embeddings.shape)
data2 = tf.placeholder(shape = (batchSize,max_sequence_length,numDimensions+char_dimensions), dtype=tf.float32)
data2 = tf.concat([data, char_embeddings], axis=2)

(100, 20, 100)
(100, 20, 100)


In [26]:
fw_cell3 = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True) for _ in range(2)])
bw_cell3 = tf.nn.rnn_cell.MultiRNNCell([ tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True) for _ in range(2)])

In [27]:
outputs,value2 = tf.nn.bidirectional_dynamic_rnn(fw_cell3, bw_cell3,data2,dtype=tf.float32)

In [28]:
#print(outputs)

outputs = tf.concat(outputs, 2)
#print(outputs.shape)

In [29]:
weight = tf.Variable(tf.truncated_normal([lstmUnits*2,numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[batchSize,max_sequence_length,numClasses]))

In [30]:
prediction = (tf.tensordot(outputs, weight, axes=((2,),(0,))) + bias)
prediction_arg = tf.argmax(prediction,axis=2)
label_arg = tf.argmax(labels,axis=2)
#false_positives
false_positives = tf.metrics.false_positives(predictions=prediction_arg, labels=label_arg)
false_negatives = tf.metrics.false_negatives(predictions=prediction_arg, labels=label_arg)
true_positives = tf.metrics.true_positives(predictions=prediction_arg, labels=label_arg)
true_negatives = tf.metrics.true_negatives(predictions=prediction_arg, labels=label_arg)
prec = tf.metrics.precision(predictions=prediction_arg, labels=label_arg)
rec = tf.metrics.recall(predictions=prediction_arg, labels=label_arg)
#false_positives = 1
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
with tf.Session() as sess:
    #tf.initialize_all_variables().run()
    sess.run(init_g)
    sess.run(init_l)
    nextBatch, nextBatchLabels, nextChars = getTrainBatch(2)
        #print(j/2)
    pred, pa, la, fp, fn, tp, tn = sess.run([prediction, prediction_arg, label_arg, false_positives, false_negatives, true_positives, true_negatives], {input_data: nextBatch, labels: nextBatchLabels, char_input_data: nextChars})
    print(pa.shape, la.shape, fp, fn, tp, tn)
    #a = pred[0]
    #b = nextBatchLabels
   # print(a.shape,b.shape)
    #print(a.shape)
    #print(np.argmax(a, axis=2)[0])
    #print(np.argmax(b, axis=2)[0])
    #print(np.logical_and(np.argmax(a, axis=2)[0], np.argmax(b, axis=2)[0] ))
    #print((np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)))[0])
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)), axis=1))
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)), axis=0))
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2))))
correctPred = tf.equal(tf.argmax(prediction,axis=2), tf.argmax(labels,axis=2))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))


(100, 20) (100, 20) (22.0, 22.0) (135.0, 135.0) (22.0, 22.0) (1821.0, 1821.0)


In [31]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [32]:
#with tf.Session() as sess:
#    sess.run(tf.global_variables_initializer())#

#    o,w,b,p = sess.run([outputs,weight,bias,prediction],feed_dict={input_data: X_train[:100], labels: y_train[:100]})
#    print(np.array(o).shape,np.array(w).shape,np.array(b).shape,np.array(p).shape)

In [33]:
import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [34]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
sess.run(init_g)
sess.run(init_l)

for i in range(epochs):
    print("epoch:  "+str(i))
    #Next Batch of reviews
    losses = 0
    for j in range(len(X_train)//batchSize):
        nextBatch, nextBatchLabels, nextChars = getTrainBatch(j);
        #print(j/2)
        ll, _,pred,lab = sess.run([loss, optimizer,prediction,labels], {input_data: nextBatch, labels: nextBatchLabels, char_input_data: nextChars})
        print(ll)
        losses += ll

    #Write summary to Tensorboard
        if (j % 5 == 0):
            summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels, char_input_data: nextChars})
            writer.add_summary(summary, j)

        #Save the network every 10,000 training iterations
        if (j % 50 == 0 and j != 0):
            save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
            print("saved to %s" % save_path)
    print(losses)
writer.close()

epoch:  0
0.47051436
0.28845656
0.26291314
0.17598605
0.15089364
0.13974608
0.1374718
0.13329159
0.13983075
0.12907177
0.1208552
0.121448
0.14193553
0.13944641
0.1402088
0.1424089
0.17134576
0.12836084
0.13334735
0.10248407
0.11797108
0.10148063
0.13616812
0.1164169
0.10221283
0.12358316
0.11421058
0.10240166
0.10386746
0.104171716
0.09735054
0.10067481
0.117694445
0.114283726
0.10272866
0.09734253
0.090941206
0.09508572
0.0835343
0.08833467
0.09679099
0.06772909
0.08993776
0.09574391
0.09765942
0.0921088
0.081579305
0.077173874
0.08306076
0.07561971
0.13100271
saved to models/pretrained_lstm.ckpt-0
0.11637102
0.13557567
0.092320554
0.10761876
0.102731235
0.10587241
0.078667805
0.10571417
0.08514621
0.09793065
0.090119995
0.09726402
0.109728284
0.073207624
0.08126083
0.0846416
0.07897229
0.08501677
0.11323842
0.08285716
0.09614566
0.09297543
0.099504545
0.06456216
0.06882134
0.08193846
0.08409824
0.0706199
0.06448699
0.07311659
0.07349131
0.07465343
0.07027827
0.066766694
0.066139214
0

0.06375275
0.078715704
0.078384906
0.06153666
0.0710481
0.040007986
0.05973678
0.040533215
0.038818672
0.049738247
0.05860888
0.08617588
0.07868852
0.08379135
0.082726285
0.050488226
0.068441115
0.05657608
0.053296585
0.040728055
0.056565344
8.659778069704771
epoch:  5
0.06295545
0.06425438
0.04598256
0.043067634
0.053641226
0.060260516
0.042013336
0.04646174
0.062252067
0.041488376
0.036517646
0.06181047
0.05228867
0.06356342
0.05596576
0.081834495
0.074313365
0.054896597
0.061436556
0.04551567
0.05398053
0.047426924
0.053197708
0.05059548
0.048740014
0.04713599
0.060294688
0.044441026
0.04861605
0.051803727
0.05443354
0.0505364
0.07775758
0.07165908
0.0725974
0.05506374
0.049909007
0.06117126
0.048470803
0.048811104
0.050489135
0.044970367
0.051950425
0.0471977
0.06176563
0.057399128
0.04652334
0.052617952
0.052776
0.044446785
0.07186432
saved to models/pretrained_lstm.ckpt-5
0.06609795
0.07637083
0.05571607
0.061343193
0.060729727
0.06988373
0.060086276
0.07755919
0.054794334
0.0655

0.058705475
saved to models/pretrained_lstm.ckpt-9
0.064014904
0.060785238
0.06360026
0.071961276
0.08934366
0.07165074
0.07456429
0.06012278
0.04915014
0.05331335
0.055963743
0.058087915
0.048998963
0.06406159
0.059944227
0.053153172
0.0661457
0.061593346
0.07028587
0.08505402
0.08127849
0.06755011
0.07931491
0.04506794
0.06662876
0.041452497
0.046530236
0.0501255
0.059536323
0.090693876
0.08528681
0.08240134
0.0864729
0.055147264
0.06557536
0.053139284
0.056794807
0.044111878
0.06024473
9.26053374633193


In [35]:
test_batches = 25
fin_pred = np.zeros([test_batches*100, 20, 2])
jj = 0
for j in range(test_batches):
    jj = j * 100
    nextBatch, nextBatchLabels, nextChars = getTestBatch(j);
    pred, acc, tp, tn, fp, fn, p, r = sess.run([prediction, accuracy, true_positives, true_negatives, false_positives, false_negatives, prec, rec], {input_data: nextBatch, labels: nextBatchLabels, char_input_data: nextChars})
    
    fin_pred[jj:jj+100] = pred

print(tp,tn,fp,fn,p,r)
    #    for i in range(10):

#        print(pred[i][:6])
#        print(nextBatchLabels[i][:6])
#        print('----------')

(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(3558.0, 3558.0) (44876.0, 44876.0) (661.0, 661.0) (905.0, 905.0) (0.8452765, 0.8433278) (0.7961233, 0.7972216)


In [36]:
prediction_analysis = {'original_text': [], 'pred_original_text': [], 'body': []}
#for i in range(len(y_test)):
for i in range((test_batches*100)):
    body = word_tokenize(str(raw_data[train_len:].iloc[i]['body']))
    lab_index_test = np.where(np.argmax(y_test, axis=2)[i]==1)[0]
    lab_index_pred = np.where(np.argmax(fin_pred, axis=2)[i]==1)[0]
    original_text = []
    pred_original_text = []
    for j in lab_index_test:
        original_text.append(body[j])
    for j in lab_index_pred:
        pred_original_text.append(body[j])

    prediction_analysis['original_text'].append(original_text)
    prediction_analysis['pred_original_text'].append(pred_original_text)
    prediction_analysis['body'].append(body)

In [37]:
prediction_analysis_pandas = pd.DataFrame.from_dict(prediction_analysis)

In [38]:
tru_pos = 0
fals_pos = 0 
fals_neg = 0
true_pos = []
false_pos = []
false_neg = []
def check_tp(original_text, pred_original_text):
    global tru_pos, fals_pos, fals_neg
    #print((set(original_text)))
    #print((set(pred_original_text)))
    og = set(original_text)
    pr = set(pred_original_text)
    res = og.intersection(pr)
    true_pos.append(res)
    
    false_pos.append(pr - res)
    false_neg.append(og - res)
    tru_pos += len((set(original_text).intersection(pred_original_text)))
    fals_pos += len(pr -res)
    fals_neg += len(og - res)
    
for i in range(2500):
    check_tp(prediction_analysis_pandas.iloc[i]['original_text'], prediction_analysis_pandas.iloc[i]['pred_original_text'])

In [39]:
pp = tru_pos/ (tru_pos + fals_pos)
rr = tru_pos/ (tru_pos + fals_neg)
tru_pos, fals_pos, fals_neg, pp, rr, (2*pp*rr)/(pp+rr)

(3556, 661, 880, 0.8432534977472137, 0.8016230838593328, 0.8219114757887439)

In [40]:
prediction_analysis_pandas['false_positives'] = false_pos
prediction_analysis_pandas['true_positives'] = true_pos
prediction_analysis_pandas['false_negatives'] = false_neg

In [43]:
prediction_analysis_pandas.to_csv('/home/pratik/Desktop/haptik/chatbot_ner/pratik_ner/models/LSTM/lstm_word_char_embeddings/pos_prediction.csv')